In [6]:
import pandas as pd
import requests

def get_passage(id, link):
    try:
        html = requests.get(link).text
        with open(f'../passages/{id}.html', 'w') as f:
            f.write(html)
    except:
        print(f'Failed to get {id}')


with open('../New_Medium_Data.csv') as f:
    df = pd.read_csv(f)
    df['vector'] = [ x for x in df['vector'].apply(lambda x: x[1:-1].split(',')) ]
    df['vector'] = df['vector'].apply(lambda x: [float(i) for i in x])

    for i, row in df.iterrows():
        get_passage(row['id'], row['link'])

    df.to_csv('../New_Medium_Data.csv', index=False)
    

Failed to get 241
Failed to get 242
